<a href="https://colab.research.google.com/github/marinathomas/SentimentAnalysisHN/blob/master/2017.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
from google.cloud import bigquery
import pandas as pd
import os
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
root_path = "/content/gdrive/Shared drives/HackerNews:SentimentAnalysis/"
credential_path = root_path+"hackernews-v2-14382f16e9b5.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

In [0]:
client = bigquery.Client()
hn_dataset_ref = client.dataset('hacker_news', project='bigquery-public-data')
hn_dset = client.get_dataset(hn_dataset_ref)

In [0]:
csv_file = root_path +  "hackernews_full/2017.csv"
stories = pd.read_csv(csv_file, index_col=0) 

In [10]:
csv_file = root_path +  "hackernews_full/coments_2017.csv"
comments_temp_2017 = pd.read_csv(csv_file, index_col=0, header=0)
comments_temp_2017.shape 

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(2295314, 14)

In [11]:
csv_file = root_path +  "hackernews_full/coments_2018.csv"
comments_temp_2018 = pd.read_csv(csv_file, index_col=0, header=0)
comments_temp_2018.shape

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(2323562, 14)

In [0]:
all_comments = pd.concat([comments_temp_2017, comments_temp_2018], ignore_index=True)

In [0]:
stories = stories.loc[stories['descendants']>0]

In [14]:
stories.shape

(4933, 14)

In [15]:
all_comments.shape

(4618876, 14)

In [0]:
comments_2017 = pd.DataFrame()

In [101]:
for index, story in stories.iterrows():
  story_comments=pd.DataFrame()
  comment_ids = []
  id = story['id'] #13292636 #
  print(id)
  count = 0;
  story_comments = all_comments.loc[all_comments['parent']== id]
  comment_ids = story_comments['id'].to_numpy()
  #print(comment_ids)
  #print(len(story_comments))
  #print(story['descendants'])
  while len(story_comments) < story['descendants']:
    id = comment_ids[count]
    temp = all_comments.loc[all_comments['parent']== id]
    story_comments = pd.concat([story_comments, temp])
    comment_ids = np.concatenate((comment_ids, temp['id'].to_numpy()))
    count += 1
    #print(len(temp))
    #print(len(comment_ids))
  #break;
  #print(comment_ids)
  
comments_2017 = pd.concat([comments_2017,story_comments])

13293894
13294375
13295049
13295057
13295998
13296297
13296863
13297192
13298843
13301479
13302725
13303346
13304193
13304291
13304390
13305005
13308093
13308150
13308263
13308523
13308662
13308820
13308857
13308875
13308977
13309025
13309390
13309724
13309838
13310834
13310974
13311064
13313156
13314087
13314370
13314409
13315576
13315991
13316467
13317067
13317148
13317718
13317907
13317978
13318093
13318289
13318528
13318624
13319122
13319371
13319814
13320359
13320370
13320638
13320994
13321024
13321586
13322062
13323115
13323240
13324460
13324490
13324493
13324971
13325632
13325670
13325797
13326109
13326189
13327461
13328843
13328865
13329708
13329836
13331968
13332015
13333185
13335042
13335442
13335554
13336257
13336686
13336954
13337861
13338592
13339093
13339877
13340462
13340618
13341156
13341621
13341665
13341677
13342341
13343075
13343184
13343276
13344291
13344747
13344808
13345429
13346903
13347326
13348124
13349076
13349625
13350508
13351064
13351326
13351434
13351973
1

IndexError: ignored

In [0]:
print(comments_2017.shape)